In [ ]:
import pandas as pd
import numpy as np
import pywt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Load dataset
mitbih_df = pd.read_csv('../datasets/mitbih_100_signals.csv')
nstdb_em_df = pd.read_csv('../datasets/nstdb_em_signal.csv')
nstdb_ma_df = pd.read_csv('../datasets/nstdb_ma_signal.csv')
nstdb_bw_df = pd.read_csv('../datasets/nstdb_bw_signal.csv')

## Preprocessing

In [ ]:
def normalize_signal(signal):
    # Normalisasi ke rentang [-1, 1]
    return 2 * (signal - min_global) / range_global - 1

def denormalize_signal(signal):
    # Kembalikan ke skala asli
    return (signal + 1) * range_global / 2 + min_global

# Normalisasi
clean_signal_asli = mitbih_df['MLII'].values
min_global = np.min(clean_signal_asli)
max_global = np.max(clean_signal_asli)
range_global = max_global - min_global

clean_signals_norm = normalize_signal(clean_signal_asli)
bw_signals_norm = normalize_signal(nstdb_bw_df['noise1'].values)
ma_signals_norm = normalize_signal(nstdb_ma_df['noise1'].values)
em_signals_norm = normalize_signal(nstdb_em_df['noise1'].values)

In [88]:
WINDOW_SIZE = 512
WAVELET_TYPE = 'db4'
WAVELET_LEVEL = 4

def create_windows(signal, window_size):
    windows = []
    for i in range(0, len(signal) - window_size + 1, window_size):
        windows.append(signal[i:i + window_size])
    return np.array(windows)

# Potong sinyal agar panjangnya sama untuk semua dataset
min_len = min(len(clean_signals_norm), len(bw_signals_norm), len(ma_signals_norm), len(em_signals_norm))
clean_windows = create_windows(clean_signals_norm[:min_len], WINDOW_SIZE)
bw_windows = create_windows(bw_signals_norm[:min_len], WINDOW_SIZE)
ma_windows = create_windows(ma_signals_norm[:min_len], WINDOW_SIZE)
em_windows = create_windows(em_signals_norm[:min_len], WINDOW_SIZE)

In [89]:
# Gabungkan semua data berderau untuk pelatihan
noisy_windows_train = np.concatenate([bw_windows, ma_windows, em_windows], axis=0)
clean_windows_train = np.concatenate([clean_windows] * 3, axis=0)

# Fungsi untuk transformasi wavelet
def transform_windows(windows, wavelet, level):
    all_coeffs, all_slices = [], []
    for window in windows:
        coeffs = pywt.wavedec(window, wavelet, level=level)
        coeffs_arr, slices = pywt.coeffs_to_array(coeffs)
        all_coeffs.append(coeffs_arr)
        all_slices.append(slices) # Simpan slice info untuk rekonstruksi
    return np.array(all_coeffs), all_slices[0] # Slice info sama untuk semua window

X_train, slice_info = transform_windows(noisy_windows_train, WAVELET_TYPE, WAVELET_LEVEL)
y_train, _ = transform_windows(clean_windows_train, WAVELET_TYPE, WAVELET_LEVEL)

print(f"Data telah disegmentasi dan ditransformasi.")
print(f"Bentuk data training (X_train): {X_train.shape}")

Data telah disegmentasi dan ditransformasi.
Bentuk data training (X_train): (3807, 538)


## Model Wavelnet + MLP

In [90]:
input_dim = X_train.shape[1]

model = Sequential([
    Dense(128, activation='relu', input_shape=(input_dim,)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(input_dim) # Layer output linear
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 128)            │        68,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 538)            │        69,402 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 154,970 (605.35 KB)

 Trainable params: 154,970 (605.35 KB)

 Non-trainable params: 0 (0.00 B)

In [91]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0251 - val_loss: 0.0133
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0109 - val_loss: 0.0122
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0101 - val_loss: 0.0121
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0102 - val_loss: 0.0120
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0100 - val_loss: 0.0120
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0102 - val_loss: 0.0120
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0099 - val_loss: 0.0121
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0101 - val_loss: 0.0120
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0101 - val_loss: 0.0121
Epoch 10/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0099 - val_loss: 0.0120
Epoch 11/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0102 - val_loss: 0.0121
Epoch 12/100
108/108 ━━━━━━━━━━━━━━━━━━━

## Denoising dan Evaluasi

In [92]:
noisy_windows_for_eval = {
    "Baseline Wander": bw_windows,
    "Motion Artifact": ma_windows,
    "Electrode Motion": em_windows
}

def calculate_rmse(clean, denoised):
    return np.sqrt(np.mean((denoised - clean) ** 2))
def calculate_prd(clean, denoised):
    return 100 * np.sqrt(np.sum((denoised - clean) ** 2) / np.sum(clean ** 2))

for noise_type, noisy_windows in noisy_windows_for_eval.items():
    # 1. Transformasi wavelet pada data uji
    X_eval, _ = transform_windows(noisy_windows, WAVELET_TYPE, WAVELET_LEVEL)

    # 2. Prediksi koefisien bersih menggunakan model
    predicted_coeffs_flat = model.predict(X_eval)

    # 3. Rekonstruksi sinyal dari koefisien yang diprediksi
    reconstructed_signals_norm = []
    for coeffs_flat in predicted_coeffs_flat:
        coeffs_structured = pywt.array_to_coeffs(coeffs_flat, slice_info, output_format='wavedec')
        reconstructed_signal = pywt.waverec(coeffs_structured, WAVELET_TYPE)
        reconstructed_signals_norm.append(reconstructed_signal[:WINDOW_SIZE])

    reconstructed_signals_norm = np.array(reconstructed_signals_norm)

    # 4. De-normalisasi sinyal untuk evaluasi
    denoised_signals_original = denormalize_signal(reconstructed_signals_norm)
    clean_eval_signals_original = denormalize_signal(clean_windows) # De-normalisasi data bersih juga

    # 5. Hitung skor
    rmses = [calculate_rmse(c, d) for c, d in zip(clean_eval_signals_original, denoised_signals_original)]
    prds = [calculate_prd(c, d) for c, d in zip(clean_eval_signals_original, denoised_signals_original)]

    print(f"Hasil Akhir untuk: {noise_type}")
    print(f"Rata-rata RMSE: {np.mean(rmses):.4f}")
    print(f"Rata-rata PRD : {np.mean(prds):.4f}% \n")


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Hasil Akhir untuk: Baseline Wander
Rata-rata RMSE: 0.1917
Rata-rata PRD : 53.6933% 

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Hasil Akhir untuk: Motion Artifact
Rata-rata RMSE: 0.1915
Rata-rata PRD : 53.6433% 

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Hasil Akhir untuk: Electrode Motion
Rata-rata RMSE: 0.1915
Rata-rata PRD : 53.6974% 

